In [19]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
from sklearn.metrics import classification_report
import warnings 
import itertools
import plotly.graph_objects as go
import numpy as np
warnings.filterwarnings("ignore")
import seaborn as sns

In [20]:
# ### Definitions
path_data_dir = '../data/'

list_assets = ["PETR3.SA","PRIO3.SA", "VALE3.SA", "GGBR3.SA", "ABCB4.SA", "ITUB3.SA", "FLRY3.SA", "RADL3.SA"]

relevant_cols = ['Date', 'Close', 'Volume']




In [22]:
n_prev_meta = 2
asset = list_assets[0]
cont_tbl_train = pd.read_csv(path_data_dir + f"processed/train_contingency_table_price_history_{asset.replace('.', '_')}_meta_range({n_prev_meta})_dataset_ffill.csv", index_col=0, header=[i for i in range(n_prev_meta)])
cont_tbl_test = pd.read_csv(path_data_dir + f"processed/test_contingency_table_price_history_{asset.replace('.', '_')}_meta_range({n_prev_meta})_dataset_ffill.csv", index_col=0, header=[i for i in range(n_prev_meta)])

In [41]:
# find all possible meta in train and test
set_meta = np.unique(
    np.append(
        cont_tbl_train.index.to_numpy(),
        cont_tbl_test.index.to_numpy()
        )
    )

max = set_meta.max()
min = set_meta.min()

# crete all combinations of "metas"
combinations = [(str(i),str(j)) for i in set_meta for j in set_meta]

# unseen combinations
unseen_comb_train = [col for col in combinations if col not in cont_tbl_train.columns]
unseen_comb_test = [col for col in combinations if col not in cont_tbl_test.columns]

# add unseen in contigency table
cont_tbl_train[unseen_comb_train] = 0
cont_tbl_test[unseen_comb_test] = 0

In [42]:
set_meta

array([-3, -2, -1,  0,  1,  2,  3])

In [43]:
seen_comb_test = cont_tbl_test.columns

for comb in seen_comb_test:
    
    print(comb)
    

    x = cont_tbl_train[comb].index
    y = cont_tbl_train[comb].values
    
    fig = go.Figure(data=[go.Bar(
            x=x, y=y,
            text=y,
         
        )])

    fig.show()
    
    
    x = cont_tbl_test[comb].index
    y = cont_tbl_test[comb].values
    
    fig = go.Figure(data=[go.Bar(
            x=x, y=y,
            text=y,
         
        )])

    fig.show()
    
    print('############################################\n\n\n')

('-3', '-3')


############################################



('-3', '-1')


############################################



('-2', '-3')


############################################



('-2', '-2')


############################################



('-2', '-1')


...

############################################



('3', '-2')


############################################



('3', '-1')


############################################



('3', '0')


############################################



('3', '1')


############################################





In [28]:
cont_tbl_train

Meta -1 -3       -2                -1       ...  3 -3       -2    -1     0  1
Meta -2 -2 -1  0 -3  -2  -1  0  1  -2   -1  ...  3  1  2  3  2  3  2  3  3  3
Meta                                        ...                              
-3       0  0  0  0   3   1  0  0   0    2  ...  0  0  0  0  0  0  0  0  0  0
-2       4  3  1  5  79  31  5  0  12   42  ...  0  0  0  0  0  0  0  0  0  0
-1       0  0  0  3  36  26  9  1  54  475  ...  0  0  0  0  0  0  0  0  0  0
 0       0  0  0  0   2   2  0  0   9  100  ...  0  0  0  0  0  0  0  0  0  0
 1       0  0  0  0   0   0  0  0   0    1  ...  0  0  0  0  0  0  0  0  0  0
 2       0  0  0  0   0   0  0  0   0    1  ...  2  0  0  0  0  0  0  0  0  0
 3       0  0  0  0   0   0  0  0   0    0  ...  1  0  0  0  0  0  0  0  0  0

[7 rows x 39 columns]

In [29]:
comb

('1', '3')